# UDI-DI 관련 데이터 탐색

In [ ]:
from typing import Tuple, List, Dict, Any
import polars as pl
import pandas as pd
from pprint import pprint
import sys
from pathlib import Path
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader
from code.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values

from code.utils import process_lazyframe_in_chunks

In [2]:
# 시스템의 논리적 CPU 코어 수를 기반으로 워커 수 결정
cpu_core = psutil.cpu_count(logical=True)

print(f"사용할 워커 수: {cpu_core}")

# 시스템의 전체 메모리 정보를 가져옴
mem = psutil.virtual_memory()

print(f"총 메모리: {mem.total / (1024**3):.2f} GB")
print(f"사용 가능 메모리: {mem.available / (1024**3):.2f} GB")
print(f"사용 중인 메모리: {mem.used / (1024**3):.2f} GB")
print(f"메모리 사용률: {mem.percent}%")

사용할 워커 수: 64
총 메모리: 125.14 GB
사용 가능 메모리: 91.93 GB
사용 중인 메모리: 33.20 GB
메모리 사용률: 26.5%


## 데이터 불러오기

In [3]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'maude.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /home/dataiku/eric/Sparta/Project3/data/maude.parquet 로딩 중... (adapter=polars)


In [4]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR/'udi.parquet',
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /home/dataiku/eric/Sparta/Project3/data/udi.parquet 로딩 중... (adapter=polars)


In [5]:
udi_second_pattern = [r'^identifiers_[1-9]\d*_id$']
udi_all_pattern = [r'^identifiers_\d+_id$']
udi_pattern = udi_all_pattern

udi_cols = get_pattern_cols(udi_lf, udi_pattern)
len(udi_cols)

132

In [6]:
cols_group = {
    'udi': udi_cols
}

udi_udi_unique = get_unique_by_cols_safe(
    udi_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor = 1
)['udi']

문자열 길이 통계를 실제 데이터로부터 추정 중...
  문자열 길이 통계:
    - 평균(mean): 13.9자
    - 중앙값(median): 14.0자
    - 75%ile: 14.0자
    - 90%ile: 14.0자
    - 최댓값: 23자
  → 사용할 크기(p75): 14.0자

=== 메모리 기반 자동 임계값 계산 ===
사용 가능한 메모리: 91.29 GB
안전 사용 메모리 (30%): 27.39 GB
예상 바이트/고유값: 106 bytes
  - 문자열 데이터: 28 bytes
  - str 오버헤드: 50 bytes
  - set 오버헤드: 28 bytes
계산된 최대 고유값: 277,414,850개



Extracting unique values:   0%|          | 0/1 [00:00<?, ?it/s]

udi: 6,336,862개의 고유값 (예상 메모리: 640.6 MB)


Extracting unique values: 100%|██████████| 1/1 [00:05<00:00,  5.32s/it]

  ✓ udi 추출 완료 (실제 메모리: 636.11 MB)
    → 예상치 정확도: 100.7% (예상/실제 비율)


=== 추출 요약 ===
성공: 1/1
실패/스킵: 0/1


In [7]:
maude_udi_unique = get_unique(maude_lf, ['device_0_udi_di'])
angry_udi = maude_udi_unique - udi_udi_unique
survive_udi = maude_udi_unique & udi_udi_unique

print(f'UDI 데이터의 고유 udi 개수: {len(udi_udi_unique)}개')
print(f'MAUDE 데이터의 고유 udi 개수: {len(maude_udi_unique)}개')
print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(angry_udi)}개')
print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(survive_udi)}개')

UDI 데이터의 고유 udi 개수: 6336861개
MAUDE 데이터의 고유 udi 개수: 142453개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 30804개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 111649개


In [8]:
udi_first_unique = get_unique(udi_lf, ['identifiers_0_id'])
len(udi_first_unique)

first_second = udi_first_unique & udi_udi_unique & maude_udi_unique
sorted(first_second)[:10]

['00000000000000',
 '00016958043815',
 '00016958044256',
 '00016958046861',
 '00016958047158',
 '00016958048957',
 '00016958051971',
 '00016958052367',
 '00016958053647',
 '00016958056402']

In [9]:
sorted(angry_udi)[:100]

['(01)(10)NA',
 '(01)SW11677',
 '(01)SW12300',
 '(01)UNKNOWN',
 '+B095321804',
 '+D590135020',
 '+D950135020',
 '+M20656410',
 '+M206715716',
 '+M20674440',
 '+M20674441',
 '+M724BSMM1',
 '+M724CCB541',
 '+M724CCB551',
 '+M724MPJS1',
 '0',
 '0(01)(10)93141',
 '0(01)(21)11442',
 '0(01)(21)70620',
 '0(01)008848380',
 '0(01)100302881',
 '0(01)26202031B',
 '0+$$+75299/16D',
 '0+D66160013941',
 '0+D661H0082019',
 '0+H2028231011Z',
 '0+H20282316411',
 '0+H66860517001',
 '0+M57250600020',
 '0+M724CCB6T721',
 '00',
 '00(01)(10)5121',
 '00(01)(17)(10)',
 '00(01)(21)3051',
 '00(01)(21)3088',
 '00(01)(21)3150',
 '00(01)(21)3168',
 '00(01)(21)3170',
 '00(01)(21)3206',
 '00(01)(21)3254',
 '00(01)(21)3270',
 '00(01)(21)3306',
 '00(01)(21)3313',
 '00(01)(21)3323',
 '00(01)(21)3330',
 '00(01)(21)3333',
 '00(01)(21)3339',
 '00(01)(21)3365',
 '00(01)(21)3370',
 '00(01)(21)3401',
 '00(01)(21)3418',
 '00(01)(21)3454',
 '00(01)(21)3459',
 '00(01)(21)3485',
 '00(01)(21)3499',
 '00(01)(21)3503',
 '00(01)(21)

In [10]:
sorted(angry_udi, reverse=True)[:100]

['N',
 'MHC18223313',
 'M986CT0264010B',
 'M970120001102202',
 'M9461083000',
 'M937710344001G',
 'M776EBVTS40LP1W',
 'M766AS09850',
 'M766132623090',
 'M766108452000',
 'M74064065500',
 'M74054106040',
 'M74054100190',
 'M74054100080',
 'M74054100070',
 'M74054100050',
 'M74054100010',
 'M74040026250',
 'M74040026200',
 'M74033022050',
 'M74033020050',
 'M74030034400',
 'M74030030320',
 'M74030026280',
 'M74002270050',
 'M738A500090',
 'M738A500060',
 'M738500090',
 'M72563100010',
 'M72555335450',
 'M7253LLIS5508100',
 'M7253CS161407090',
 'M7253CS161407060',
 'M72520201020',
 'M718VPX40000',
 'M718VOX4000',
 'M701TVIL1428160I1',
 'M701TVIL1422160I1',
 'M701TVIL1418140J',
 'M701TVIL1418140I1',
 'M701TVIL1418100I1',
 'M701TVIL1416140I1',
 'M701TVIL1416120I1',
 'M701TVIL1416100I1',
 'M701TVIL1414160I1',
 'M701TVIL1414140I1',
 'M701TVIL1412120I1',
 'M701TVIL1412120D1',
 'M701TVIL1412100I1',
 'M701TVAB3480L1',
 'M701TVAB3480I1',
 'M701TVAB3480D1',
 'M701TVAB2980L1',
 'M701TVAB2980I1',
 '

In [11]:
sorted(survive_udi)[:10]

['00000000000000',
 '00000050223480',
 '00009336009480',
 '00009336009541',
 '00010939701442',
 '00015482232290',
 '00015482234324',
 '00015482235314',
 '00015482236328',
 '00015482238315']

In [12]:
maude_lf.filter(
    pl.col('device_0_udi_di').is_in(list(maude_udi_unique - angry_udi))
).select(pl.len()).collect().item() / maude_lf.select(pl.len()).collect().item() * 100

58.3198144405524

In [13]:
maude_drop_lf = maude_lf.drop(pl.col(r"^(device|mdr_text|patient)_[1-9]\d*_.*$"))
len(maude_drop_lf.collect_schema().names())

143

In [14]:
def search_condition(field: str, value: str, exact=False) -> bool:
    if exact:
        cond = pl.col(field).str.to_uppercase() == value
    else:
        cond = pl.col(field).str.to_uppercase().str.starts_with(value.upper())

    return cond

def search(lf: pl.LazyFrame, query: List[Dict[str, Any]], cols: List[str], n_rows: int = 10, transpose=False) -> pd.DataFrame:
    if not cols:
        cols = lf.collect_schema().names()
    
    cond = True
    for q in query:
        cond &= search_condition(q['field'], q['value'], q.get('exact', False))
        
    response_df = lf.select(cols).filter(cond).head(n_rows).collect().to_pandas()
    if transpose:
        response_df = response_df.transpose()
    return response_df

In [15]:
udi_select_cols = ['company_name', 'brand_name', 'version_or_model_number', 'identifiers_0_id']

maude_select_cols = ['device_0_manufacturer_d_name', 'device_0_brand_name', 'device_0_model_number', 'device_0_udi_di']

## 스코핑해서 보기

In [16]:
udi_dis = ['(01)SW11677', 'M766132623090', 'M701TVAB3480D1']

In [17]:
for udi_di in udi_dis:
    query = [
        {
            'field': 'device_0_udi_di',
            'value': udi_di,
            'exact': True
        }
    ]

    display(search(maude_drop_lf, query, maude_select_cols))

,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,None,(01)SW11677


,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,CAIRE INC.,LIBERATOR,13262309,M766132623090


,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,ENDOLOGIX SANTA ROSA,OVATION PRIME,TV-AB3480-D,M701TVAB3480D1


In [18]:
company_names = ['DEXCOM', 'CAIRE', 'ENDOLOGIX']
brand_names = ['DEXCOM G6', 'LIBERATOR', 'OVATION PRIME']

In [19]:
for c_name in company_names:
    query = [
        {
            'field': 'company_name',
            'value': c_name
        }
    ]
    display(search(udi_lf, query, udi_select_cols, n_rows=50).sort_values(by='brand_name'))

,company_name,brand_name,version_or_model_number,identifiers_0_id
45,"Dexcom, Inc.",Dexcom Discovery Sensor 3 Pack,STS-DE-003,70386270000254
12,"Dexcom, Inc.",Dexcom G4 Global Transmitter,9438-01,00386270000040
49,"Dexcom, Inc.",Dexcom G4 PLATINUM (Pediatric) Receiver with S...,STK-CR-BLU,20386270000204
21,"Dexcom, Inc.",Dexcom G4 PLATINUM (Pediatric) Receiver with S...,STK-CR-001,20386270000181
2,"Dexcom, Inc.",Dexcom G4 PLATINUM (Pediatric) Receiver with S...,STK-PR-BLU,10386270000207
16,"Dexcom, Inc.",Dexcom G4 PLATINUM (Pediatric) Receiver with S...,STK-PR-001,10386270000184
18,"Dexcom, Inc.",Dexcom G4 PLATINUM (Pediatric) Replacement Rec...,STR-PR-PNK,30386270000195
30,"Dexcom, Inc.",Dexcom G4 PLATINUM Replacement Receiver with S...,STR-DR-001,30386270000157
48,"Dexcom, Inc.",Dexcom G4 Platinum (Pedicatric) Receiver Kit,STK-KD-001,10386270000078
22,"Dexcom, Inc.",Dexcom G4 Platinum (Pedicatric) Receiver Kit,STK-KD-PNK,10386270000108


,company_name,brand_name,version_or_model_number,identifiers_0_id
23,CAIRE INC.,Companion 5,15067005,M766150670050
4,CAIRE INC.,"Eclipse 3 Assembly, Re-Manufactured",5985C-SEQ,M7665985CSEQ0
1,CAIRE INC.,Eclipse Oxygen Concentrator,5900C-SEQ,M7665900CSEQ0
39,CAIRE INC.,Eclipse Oxygen Concentrator,6900-SEQ,M7666900SEQ0
21,CAIRE INC.,Focus,AS078-1U,M766AS0781U0
8,CAIRE INC.,FreeStyle 5,AS077-1CU,M766AS0771CU0
12,CAIRE INC.,FreeStyle 5,AS077-1U,M766AS0771U0
34,CAIRE INC.,HELiOS 46,B-701652-00,M766B701652000
11,CAIRE INC.,HELiOS 46,B-702099-00,M766B702099000
28,CAIRE INC.,HOMESTYLE Aspen,SCE1000-NANB,M766SCE1000NANB0


,company_name,brand_name,version_or_model_number,identifiers_0_id
26,Endologix LLC,AFX Endovascular AAA System,BA22-60/I16-40,00818009010193
48,Endologix LLC,AFX Endovascular AAA System,BA25-110/I16-30,00818009012180
42,Endologix LLC,AFX Endovascular AAA System,BA22-120/I16-40,00818009013965
39,Endologix LLC,AFX Endovascular AAA System,BA25-40/I20-40,00818009014085
18,Endologix LLC,AFX Endovascular AAA System,BA28-100/I20-40,00818009014207
17,Endologix LLC,AFX Endovascular AAA System,A31-31/C80-O20,00818009011275
32,Endologix LLC,AFX Endovascular AAA System,BA28-100/I16-55,00818009010063
25,Endologix LLC,AFX Endovascular AAA System,BA25-80/I20-40,00818009012210
44,Endologix LLC,AFX Endovascular AAA System,BA25-70/I20-30,00818009012197
13,Endologix LLC,AFX Endovascular AAA System,I20-20/C55 SA,00818009011428


In [20]:
for c_name in company_names:
    query = [
        {
            'field': 'device_0_manufacturer_d_name',
            'value': c_name
        }
    ]
    display(search(maude_lf, query, maude_select_cols, n_rows=50).sort_values(by='device_0_manufacturer_d_name'))

,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,MT24078,00386270000583
27,"DEXCOM, INC.",DEXCOM G5 MOBILE CONTINUOUS GLUCOSE MONITORING...,9438-06,20386270000228
28,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
29,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
30,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
31,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
32,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
33,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,MT24078,00386270000583
34,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385
35,"DEXCOM, INC.",DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,9445-02,00386270000385


,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,CAIRE INC.,FREESTYLE COMFORT,AS200-3,None
26,CAIRE INC.,NEWLIFE INTENSITY 10,AS099-101,None
27,CAIRE INC.,NEWLIFE INTENSITY 10,AS099-100,None
28,CAIRE INC.,NEWLIFE INTENSITY 10,AS099-101,None
29,CAIRE INC.,COMPANION 5,15067005,None
30,CAIRE INC.,NEWLIFE ELITE,AS005-4,None
31,CAIRE INC.,NEWLIFE INTENSITY 10L,AS099-100,None
33,CAIRE INC.,COMPANION 5,15067005,None
34,CAIRE INC.,NEWLIFE INTENSITY 10,AS099-100,None
35,CAIRE INC.,FREESTYLE 3,AS095-103,None


,device_0_manufacturer_d_name,device_0_brand_name,device_0_model_number,device_0_udi_di
0,ENDOLOGIX,AFX2,BEA28-80/I20-40,00818009014412
25,ENDOLOGIX,AFX2,BEA28-120/I16-40,00818009014443
27,ENDOLOGIX,AFX2,BEA22-90/I16-30,00818009015136
29,ENDOLOGIX,OVATION PRIME,TV-AB3480-E,None
30,ENDOLOGIX,AFX2,BEA28-100/I16-40,00818009014450
31,ENDOLOGIX,AFX,BA25-80/I16-40,None
32,ENDOLOGIX,AFX,BA28-90/I20-30,00818009012630
33,ENDOLOGIX,AFX,BA28-100/I16-40,None
34,ENDOLOGIX,AFX,A34-34/C100-O20 V,00818009014207
48,ENDOLOGIX,AFX2,BEA28-80/I16-40,00801009014467


In [21]:
for b_name in brand_names:
    query = [
        {
            'field': 'brand_name',
            'value': b_name
        }
    ]
    display(search(udi_lf, query, udi_select_cols, 100))

,company_name,brand_name,version_or_model_number,identifiers_0_id
0,"Dexcom, Inc.",Dexcom G6 Receiver,STK-FR-001,00386270003225
1,"Dexcom, Inc.",Dexcom G6 Sensor Applicator Assembly,9500-45,00386270000255
2,"Dexcom, Inc.",Dexcom G6 iOS,SW11677,00386270000804
3,"Dexcom, Inc.",Dexcom G6 Android,SW11678,00386270000811
4,"Dexcom, Inc.",Dexcom G6 Transmitter,9445-24,00386270001627
5,"Dexcom, Inc.",Dexcom G6 Sensor Kit (1) Medicare,STS-OM-001,80386270000251
6,"Dexcom, Inc.",Dexcom G6 Receiver,STK-FM-001,00386270003232
7,"Dexcom, Inc.",Dexcom G6 Receiver,STK-FE-001,00386270002860
8,"Dexcom, Inc.",Dexcom G6 Professional System,STP-PR-001,00386270001092
9,"Dexcom, Inc.",Dexcom G6 Receiver Kit,STK-OR-001,00386270000590


,company_name,brand_name,version_or_model_number,identifiers_0_id
0,CAIRE INC.,Liberator,20606186SM,M76620606186SM0
1,CAIRE INC.,Liberator,20651170,M766206511700
2,CAIRE INC.,Liberator,14454978,M766144549780
3,Cook Vandergrift Inc.,Liberator Beacon Tip,G26550,00827002265509
4,CAIRE INC.,Liberator,20606186,M766206061860
...,...,...,...,...
68,CAIRE INC.,Liberator,13256591,M766132565910
69,CAIRE INC.,Liberator,13259863,M766132598630
70,CAIRE INC.,Liberator,13263782,M766132637820
71,CAIRE INC.,Liberator,14454978SM,M76614454978SM0


,company_name,brand_name,version_or_model_number,identifiers_0_id
0,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL141680E1
1,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Aortic Body,M701TVAB2380E1
2,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Extension,M701TVEX121245E1
3,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL1410120E1
4,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL1414120E1
5,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL1412100E1
6,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL1416100E1
7,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL1416140E1
8,"Trivascular, Inc.",Ovation Prime Abdominal Stent Graft System,Iliac Limb,M701TVIL141480E1
9,"Trivascular, Inc.",Ovation Prime Abominal Stent Graft System,Iliac Limb,M701TVIL1412120E1


In [22]:
maude_select_cols = ['device_0_manufacturer_d_name', 'device_0_brand_name', 'device_0_model_number', 'device_0_udi_di']
maude_drop_lf.select(maude_select_cols).filter(
    pl.col('device_0_udi_di') == '(01)SW11677'
).head(10).collect().to_pandas().transpose()

,0
device_0_manufacturer_d_name,"DEXCOM, INC."
device_0_brand_name,DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM
device_0_model_number,None
device_0_udi_di,(01)SW11677


In [23]:
maude_drop_lf.select(maude_select_cols).filter(
    pl.col('device_0_brand_name').str.to_uppercase().str.starts_with('OVATION PRIME')
).head(100).collect().to_pandas().transpose()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
device_0_manufacturer_d_name,ENDOLOGIX SANTA ROSA,ENDOLOGIX,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,...,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA,ENDOLOGIX SANTA ROSA
device_0_brand_name,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,...,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME,OVATION PRIME
device_0_model_number,TV-IL1414140-E,TV-AB3480-E,TV-AB3480-E,TV-AB2680-D,TV-AB2980-I,TV-AB2680-E,TV-AB3480-E,TV-IL1416140-E,TV-IL1412120-D,TV-EX181845-E,...,TV-AB3480-D,TV-AB2080-E,TV-AB3480-D,TV-AB2380-E,TV-IL1418140-E,None,TV-AB3480-E,TV-AB3480-E,TV-AB2980-E,TV-IL1414100-E
device_0_udi_di,M701TVIL1414140E1,None,None,None,M701TVAB2980I1,M701TVAB2680E1,None,None,M701TVIL1412120D1,M701TVEX181845E1,...,None,M701TVAB2080E1,M701TVAB3480D1,None,M701TVIL1418140E1,None,None,M701TVAB3480E1,None,None


## UDI-DI 데이터셋 이해

In [24]:
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]

identifiers_cols = get_pattern_cols(udi_lf, IDENTIFIER_PATTERNS)
len(identifiers_cols)

TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

### 가장 udi가 많은 케이스 확인

In [25]:
package_udi_lf = udi_lf.filter(
    pl.col('identifiers_131_id').is_not_null()
)

package_udi_lf.collect().to_pandas().transpose().dropna()

,0
brand_name,TRUE METRIX
commercial_distribution_status,In Commercial Distribution
company_name,"TRIVIDIA HEALTH, INC."
customer_contacts_0_email,customercare@trividiahealth.com
customer_contacts_0_phone,+1(800)803-6025
...,...
storage_0_high_value,86
storage_0_low_unit,Degrees Fahrenheit
storage_0_low_value,40
storage_0_type,Storage Environment Temperature


In [26]:
package_udi_lf.select(udi_cols).collect().to_pandas().transpose()

,0
identifiers_0_id,00021292007836
identifiers_100_id,00036800234550
identifiers_101_id,10036800234557
identifiers_102_id,00036800234536
identifiers_103_id,10036800234533
...,...
identifiers_96_id,00050428322659
identifiers_97_id,10050428322656
identifiers_98_id,00036800234543
identifiers_99_id,10036800234540


In [27]:
package_udi_set = set(package_udi_lf.select(udi_cols).collect().transpose().to_series().drop_nulls().to_list())

In [28]:
maude_udi_unique & package_udi_set

{'00021292007836'}

### Primary 컬럼 생성

In [29]:
# 가장 중요한 결측치 확인
analyze_null_values(udi_lf, ['identifiers_0_id'])


=== 결측치 분석 ===
전체 행 수: 4,903,764

identifiers_0_id                             :          0개 (  0.00%)


column,null_count,null_pct
str,u32,f64
"""identifiers_0_id""",0,0.0


In [30]:
TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

total_primaries = udi_lf.select(
    pl.sum_horizontal([pl.col(col).eq("Primary") for col in type_cols]).sum()
).collect().item()

total_primaries

4903764

In [31]:
# type과 id 컬럼들을 매칭
type_cols = get_pattern_cols(udi_lf, [r'identifiers_\d+_type'])
id_cols = get_pattern_cols(udi_lf, [r'identifiers_\d+_id'])

# type-id 쌍 만들기 (인덱스로 매칭)
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    id_col = f'identifiers_{idx}_id'
    if id_col in id_cols:
        type_id_pairs.append((type_col, id_col))


unique_primary_ids = set()

for i, (type_col, id_col) in enumerate(type_id_pairs):
    print(f"Processing {i+1}/{len(type_id_pairs)}: {type_col}...", end=" ")
    
    try:
        # 작은 쿼리로 나눠서
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            unique_primary_ids.update(ids)
            print(f"✓ Found {len(ids)} unique Primary IDs")
        else:
            print("✓ No Primary found")
            
    except Exception as e:
        print(f"✗ Error: {e}")
        continue

print(f"\n{'='*50}")
print(f"Total unique Primary IDs: {len(unique_primary_ids)}")

Processing 1/132: identifiers_0_type... ✓ Found 3802148 unique Primary IDs
Processing 2/132: identifiers_100_type... ✓ No Primary found
Processing 3/132: identifiers_101_type... ✓ No Primary found
Processing 4/132: identifiers_102_type... ✓ No Primary found
Processing 5/132: identifiers_103_type... ✓ No Primary found
Processing 6/132: identifiers_104_type... ✓ No Primary found
Processing 7/132: identifiers_105_type... ✓ No Primary found
Processing 8/132: identifiers_106_type... ✓ No Primary found
Processing 9/132: identifiers_107_type... ✓ No Primary found
Processing 10/132: identifiers_108_type... ✓ No Primary found
Processing 11/132: identifiers_109_type... ✓ No Primary found
Processing 12/132: identifiers_10_type... ✓ No Primary found
Processing 13/132: identifiers_110_type... ✓ No Primary found
Processing 14/132: identifiers_111_type... ✓ No Primary found
Processing 15/132: identifiers_112_type... ✓ No Primary found
Processing 16/132: identifiers_113_type... ✓ No Primary found
Proc

In [32]:
# Primary인 id를 추출
udi_with_primary_lf = udi_lf.with_columns(
    pl.coalesce([
        pl.when(pl.col(type_col).eq("Primary"))
        .then(pl.col(id_col))
        for type_col, id_col in type_id_pairs
    ]).alias('primary_udi_di')
)

# 확인
udi_with_primary_lf.head(5).collect()

brand_name,catalog_number,commercial_distribution_end_date,commercial_distribution_status,company_name,customer_contacts_0_email,customer_contacts_0_ext,customer_contacts_0_phone,customer_contacts_100_email,customer_contacts_100_phone,customer_contacts_101_email,customer_contacts_101_phone,customer_contacts_102_email,customer_contacts_102_phone,customer_contacts_103_email,customer_contacts_103_phone,customer_contacts_104_email,customer_contacts_104_phone,customer_contacts_105_email,customer_contacts_105_phone,customer_contacts_106_email,customer_contacts_106_phone,customer_contacts_107_email,customer_contacts_107_phone,customer_contacts_108_email,customer_contacts_108_phone,customer_contacts_109_email,customer_contacts_109_phone,customer_contacts_10_email,customer_contacts_10_phone,customer_contacts_110_email,customer_contacts_110_phone,customer_contacts_111_email,customer_contacts_111_phone,customer_contacts_112_email,customer_contacts_112_phone,customer_contacts_113_email,…,storage_4_high_value,storage_4_low_unit,storage_4_low_value,storage_4_special_conditions,storage_4_type,storage_5_high_unit,storage_5_high_value,storage_5_low_unit,storage_5_low_value,storage_5_special_conditions,storage_5_type,storage_6_high_unit,storage_6_high_value,storage_6_low_unit,storage_6_low_value,storage_6_special_conditions,storage_6_type,storage_7_high_unit,storage_7_high_value,storage_7_low_unit,storage_7_low_value,storage_7_special_conditions,storage_7_type,storage_8_high_unit,storage_8_high_value,storage_8_low_unit,storage_8_low_value,storage_8_special_conditions,storage_8_type,storage_9_high_unit,storage_9_high_value,storage_9_low_unit,storage_9_low_value,storage_9_special_conditions,storage_9_type,version_or_model_number,primary_udi_di
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Medtronic Reusable Instrument""",null,null,"""In Commercial Distribution""","""MEDTRONIC SOFAMOR DANEK, INC.""","""Corporate.UDI@medtronic.com""",null,"""+1(800)633-8766""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""X0613995""","""00643169405226"""
"""CD HORIZON® Spinal System""",null,null,"""In Commercial Distribution""","""MEDTRONIC SOFAMOR DANEK, INC.""","""Corporate.UDI@medtronic.com""",null,"""+1(800)633-8766""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""25740011270""","""00643169415027"""
"""CD HORIZON Spinal System""",null,null,"""In Commercial Distribution""","""MEDTRONIC SOFAMOR DANEK, INC.""","""Corporate.UDI@medtronic.com""",null,"""+1(800)633-8766""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""661101110""","""00643169600355"""
"""Sublime Sensation""",null,null,"""In Commercial Distribution""","""TRULIFE LIMITED""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

## Meta Data

In [33]:
import polars as pl

def process_file(input_file, output_file):
    """메모리 효율적 + 빠른 처리"""
    with open(input_file, 'rb') as f_in, \
         open(output_file, 'wb') as f_out:
        
        # 헤더
        header = f_in.readline()
        f_out.write(header.decode('euc-kr', errors='ignore').encode('utf-8'))
        
        expected = header.decode('euc-kr', errors='ignore').count('|')
        
        buffer = []
        buffer_size = 10000  # 10K 라인씩 버퍼링
        
        for line_bytes in f_in:
            line_str = line_bytes.decode('euc-kr', errors='ignore')
            
            if line_str.count('|') == expected:
                buffer.append(line_str.encode('utf-8'))
                
                if len(buffer) >= buffer_size:
                    f_out.writelines(buffer)
                    buffer = []
        
        if buffer:
            f_out.writelines(buffer)

# process_file(DATA_DIR / 'mdrfoiThru2024.txt', '00.txt')


In [34]:

# Polars lazy 로딩
meta_lf = pl.scan_csv(
    DATA_DIR /'01.txt', 
    separator='|',
    ignore_errors=True
)

total_meta_cols = meta_lf.collect_schema().names()
print(len(total_meta_cols))

null_df = analyze_null_values(meta_lf, total_meta_cols)

# 필요한 작업만 정의 (아직 실행 안됨)
# result = df.select(['컬럼1', '컬럼2']).filter(...)

# 실제 실행 - 메모리에 올림
# result.collect()

# 또는 메모리 안 쓰고 바로 저장
# result.sink_parquet('output.parquet')

# 3. 작업 정의 (메모리 안 씀)
# result = (
#     df
#     .select(['col1', 'col2', 'col3'])  # 필요한 컬럼만
#     .filter(pl.col('date') > '2024-01-01')  # 필터링
#     .group_by('category').agg(pl.col('value').sum())  # 집계
# )

# 4-a. 결과가 작으면 메모리에 올림
# final = result.collect()

# # 4-b. 결과가 크면 바로 파일로
# result.sink_parquet('result.parquet')  # 메모리 안 씀!

84

=== 결측치 분석 ===
전체 행 수: 999

EVENT_KEY                                    :        999개 (100.00%)
NUMBER_DEVICES_IN_EVENT                      :        999개 (100.00%)
NUMBER_PATIENTS_IN_EVENT                     :        999개 (100.00%)
DATE_REPORT_TO_MANUFACTURER                  :        999개 (100.00%)
MANUFACTURER_CONTACT_EXTENSION               :        999개 (100.00%)
DISTRIBUTOR_NAME                             :        999개 (100.00%)
DISTRIBUTOR_ADDRESS_1                        :        999개 (100.00%)
DISTRIBUTOR_ADDRESS_2                        :        999개 (100.00%)
DISTRIBUTOR_CITY                             :        999개 (100.00%)
DISTRIBUTOR_STATE_CODE                       :        999개 (100.00%)
DISTRIBUTOR_ZIP_CODE                         :        999개 (100.00%)
DISTRIBUTOR_ZIP_CODE_EXT                     :        999개 (100.00%)
REPORT_TO_MANUFACTURER                       :        999개 (100.00%)
MANUFACTURER_NAME                            :        999개 (100.00%)
MA

## Primary와 매칭하기

### Primary 컬럼 생성

In [1]:
# Primary인 id를 추출
udi_with_primary_lf = udi_lf.with_columns(
    pl.coalesce([
        pl.when(pl.col(type_col).eq("Primary"))
        .then(pl.col(id_col))
        for type_col, id_col in type_id_pairs
    ]).alias('primary_udi_di')
)

NameError: name 'udi_lf' is not defined

In [ ]:
udi_with_primary_lf.filter(
    pl.col('primary_udi_di').eq('00887761000872')
).head(10).collect().to_pandas()

# 커스텀 변환 함수
def uppercase_transform(lf):
    return lf.with_columns([
        pl.col("text").str.to_uppercase(),
        pl.col("name").str.to_lowercase()
    ])

# Chunk 처리
process_lazyframe_in_chunks(
    lf=data_lf,
    transform_func=uppercase_transform,
    output_path=Path('output.parquet'),
    chunk_size=500_000,
    desc="Uppercase conversion"
)

: 

In [ ]:
# 1. Primary IDs를 set으로 수집
primary_udi_unique = set()
for type_col, id_col in type_id_pairs:
    ids = (
        udi_lf
        .filter(pl.col(type_col).eq("Primary"))
        .select(pl.col(id_col))
        .unique()
        .collect()
        .to_series()
        .drop_nulls()
        .to_list()
    )
    primary_udi_unique.update(ids)

print(f"고유 Primary 개수: {len(primary_udi_unique)}")

# Primary IDs를 Polars DataFrame으로 만들고 lazy로
primary_lf = pl.LazyFrame({
    'udi_di': list(primary_udi_unique)
})

maude_udi_lf = pl.LazyFrame({
    'udi_di': list(maude_udi_unique)
})

# 매칭되는 unique UDI-DI 리스트
maude_udi_matched = (
    maude_udi_lf
    .select('udi_di')
    .unique()
    .join(
        primary_lf,
        left_on='udi_di',
        right_on='udi_di',
        how='semi'
    )
    .collect()
    .to_series()
    .to_list()
)

# 매칭 안 되는 unique UDI-DI 리스트
maude_udi_unmatched = (
    maude_udi_lf
    .select('udi_di')
    .unique()
    .join(
        primary_lf,
        left_on='udi_di',
        right_on='udi_di',
        how='anti'
    )
    .collect()
    .to_series()
    .to_list()
)

# 확인
print(f"Primary와 매칭 되는 고유 MAUDE UDI-DI: {len(maude_udi_matched):,}")
print(f"Primary와 매칭 되지 않는 고유 MAUDE UDI-DI: {len(maude_udi_unmatched):,}")
print(f"매칭률: {len(maude_udi_matched) / (len(maude_udi_unique)) * 100:.2f}%")

In [ ]:
# Unique UDI-DI만 추출
maude_unique_lf = maude_lf.select('device_0_udi_di').unique()

# 매칭되는 unique UDI-DI
maude_udi_matched_lf = (
    maude_unique_lf
    .join(
        primary_lf,
        left_on='device_0_udi_di',
        right_on='udi_di',
        how='semi'
    )
)

# 매칭 안 되는 unique UDI-DI
maude_udi_unmatched_lf = (
    maude_unique_lf
    .join(
        primary_lf,
        left_on='device_0_udi_di',
        right_on='udi_di',
        how='anti'
    )
)

# 원본 데이터를 다시 join
maude_udi_full_lf = maude_lf.join(
    maude_unique_lf,
    on='device_0_udi_di',
    how='semi'
)

maude_matched_lf = maude_lf.join(
    maude_udi_matched_lf,
    on='device_0_udi_di',
    how='semi'
)

maude_unmatched_lf = maude_lf.join(
    maude_udi_unmatched_lf,
    on='device_0_udi_di',
    how='semi'
)

# 확인
print(f"전체 MAUDE 데이터: {maude_lf.select(pl.len()).collect().item():,}")
print(f"UDI 있는 MAUDE 데이터: {maude_udi_full_lf.select(pl.len()).collect().item():,}")
print(f"Primary와 매칭 되는 MAUDE 데이터: {maude_matched_lf.select(pl.len()).collect().item():,}")
print(f"Primary와 매칭 안 되는 MAUDE 데이터: {maude_unmatched_lf.select(pl.len()).collect().item():,}")

In [ ]:
sorted(maude_udi_matched, reverse=True)[:100]

In [ ]:
# 직접 컬럼명 명시
maude_multiple_primaries_detailed = (
    maude_lf
    .select('device_0_udi_di')
    .unique()
    .join(
        primary_lf.rename({'udi_di': 'primary_udi_di'}),
        left_on='device_0_udi_di',
        right_on='primary_udi_di',
        how='inner'
    )
    # .group_by('device_0_udi_di')
    # .agg([
    #     pl.len().alias('primary_count'),
    #     pl.col('primary_udi_di').alias('matched_primaries')
    # ])
    # .filter(pl.col('primary_count') >= 2)
    # .sort('primary_count', descending=True)
)

result_detailed = maude_multiple_primaries_detailed.collect()
print(result_detailed)

In [ ]:
# 어떤 Primary들과 매칭되는지 확인
maude_multiple_primaries_detailed = (
    maude_lf
    .select('device_0_udi_di')
    .unique()
    .join(
        udi_lf,
        left_on='device_0_udi_di',
        right_on='udi_di',
        how='inner'
    )
    .group_by('device_0_udi_di')
    .agg([
        pl.col('udi_di').count().alias('primary_count'),
        pl.col('udi_di').alias('matched_primaries')
    ])
    .filter(pl.col('primary_count') >= 2)
    .sort('primary_count', descending=True)
)

result_detailed = maude_multiple_primaries_detailed.collect()
print(result_detailed)

In [ ]:
udi_dis = sorted(maude_udi_matched)[:10]

for udi_di in udi_dis:
    query = [
        {
            'field': 'device_0_udi_di',
            'value': udi_di,
            'exact': True
        }
    ]

    display(search(maude_drop_lf, query, maude_select_cols, n_rows=100))

In [ ]:
udi_dis = sorted(maude_udi_matched)[:1]

for udi_di in udi_dis:
    query = [
        {
            'field': 'identifiers_2_id',
            'value': udi_di,
            'exact': True
        }
    ]

    display(search(udi_lf, query, [*udi_select_cols, 'identifiers_2_id', 'identifiers_2_type'], n_rows=100))

In [ ]:


company_names = [
    'ALCON'
]

for c_name in company_names:
    query = [
        {
            'field': 'company_name',
            'value': c_name,
        }
    ]

    display(search(udi_lf, query, [*udi_select_cols, 'identifiers_2_id', 'identifiers_2_type'], n_rows=100))